## Libraries importation

In [1]:
# Please use these lines in every notebook you create

import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from global_variables import *

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import Delaunay, Voronoi
from copy import deepcopy
import math

from python_scripts.graphs.graphs_creation import delaunay_graph, kNN_graph
from python_scripts.neighbours_criteria.enhanced_criteria import distance_criterion_enhanced
from python_scripts.neighbours_criteria.simple_criteria import angle_criterion
from python_scripts.city.city_utils import mean_distance_to_NN, mean_distance_choice
from python_scripts.miscellaneaous.data_processing import extract_data

## Database import and data extraction

In [3]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df = extract_data(df, provider=PROVIDER, region=REGION, techno=TECHNO)
df.head()

,x,y,latitude,longitude,nom_dep,nom_com,site_2g,site_3g,site_5g
id_station_anfr,,,,,,,,,
0272290011,573458.0,6899259.0,49.18056,1.26445,Eure,Fontaine-Bellenger,1,1,0
0142290016,454237.0,6903487.0,49.18389,-0.37167,Calvados,Caen,1,1,1
0762290018,560709.0,6927372.0,49.43055,1.08028,Seine-Maritime,Rouen,1,1,1
0142290003,496750.0,6915926.0,49.31055,0.20500,Calvados,Saint-Gatien-des-Bois,1,1,0
0142290018,499454.0,6927334.0,49.41389,0.23667,Calvados,Honfleur,1,1,1


In [4]:
df_azimuth_raw = pd.read_csv("../../database/data_azimuth_freq.csv", sep=";", decimal=",", dtype={'id_station_anfr':str})

dep_codes = ['027', '076', '014', '050', '061']
df_azimuth = pd.DataFrame(columns=df_azimuth_raw.columns)
for row in df_azimuth_raw.values:
    if(row[0][0:3] in dep_codes):
        df_azimuth = pd.concat([df_azimuth,pd.DataFrame([row],columns=df_azimuth_raw.columns)],ignore_index=True)
        
df_azimuth.head()

/tmp/ipykernel_13744/138318875.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_azimuth = pd.concat([df_azimuth,pd.DataFrame([row],columns=df_azimuth_raw.columns)],ignore_index=True)


,id_station_anfr,frequency_GHz,angle_azimuth
0,0140030031,1.5,252
1,0140030329,1.0,31
2,0140030590,1.0,72
3,0140034026,1.0,256
4,0140034027,1.5,62


## City detection

In [5]:
mean_distances = mean_distance_to_NN(df[['x', 'y']], n_neighbours=N_NEIGH)

## Voronoi and Delaunay creation

In [6]:
del_G, pos = delaunay_graph(df)
print(len(del_G.edges))

4355


In [7]:
from sklearn.neighbors import NearestNeighbors
K=15
kNN_G, _ = kNN_graph(df, k=K)
coordsXY = df[['x','y']]

# for edge in kNN_G.edges: # applying gabriel
#     pt1 = edge[0]
#     pt2 = edge[1]

#     middle_point = (coordsXY.loc[pt1] + coordsXY.loc[pt2])/2

#     neigh = NearestNeighbors(radius=np.sqrt(np.sum((coordsXY.loc[pt1] - coordsXY.loc[pt2])**2, axis=0))/2)
#     neigh.fit(coordsXY.values)

#     if(len(coordsXY.iloc[neigh.radius_neighbors([middle_point], sort_results=True)[1][0][:-2]].index)>0):
#         kNN_G.remove_edges_from([edge])
# print(len(kNN_G.edges))

In [8]:
# Function to check if angle is within coverage
def is_within_coverage(station_pos, neighbor_pos, azimuth, beamwidth):
    direction_vector = neighbor_pos - station_pos
    direction_angle = (np.degrees(np.arctan2(direction_vector[1], direction_vector[0])) + 360) % 360
    min_angle = (azimuth - beamwidth / 2 + 360) % 360
    max_angle = (azimuth + beamwidth / 2 + 360) % 360

    if min_angle <= max_angle:
        return min_angle <= direction_angle <= max_angle
    else:
        return direction_angle >= min_angle or direction_angle <= max_angle

    

In [9]:
# Function to find real neighbors based on Delaunay triangulation and azimuths
def find_real_neighbors(df, df_azimuth, potential_neigh_G, pos):
    neigh_G = deepcopy(potential_neigh_G)
    edge_info = {}

    for bs_id in df.index:
        azimuths = df_azimuth.loc[df_azimuth['id_station_anfr'] == bs_id, 'angle_azimuth']
        nb_azimuth = len(azimuths)
        beamwidth = (360 / nb_azimuth) if (nb_azimuth > 1) else 120

        for [_, neigh_id] in potential_neigh_G.edges(bs_id):
            is_neighbor = False
            azimuths_info = {"station": [], "neighbor": []}

            for azimuth in azimuths:
                if is_within_coverage(pos[bs_id], pos[neigh_id], azimuth, beamwidth):
                    neigh_azimuth = df_azimuth.loc[df_azimuth['id_station_anfr'] == neigh_id, 'angle_azimuth']
                    neigh_nb_azimuth = len(neigh_azimuth)
                    beamwidth_neigh = (360 / neigh_nb_azimuth) if (neigh_nb_azimuth > 1) else 120

                    for neighbor_azimuth in neigh_azimuth:
                        if is_within_coverage(pos[neigh_id], pos[bs_id], neighbor_azimuth, beamwidth_neigh):
                            is_neighbor = True
                            azimuths_info["station"] = [bs_id, azimuth]
                            azimuths_info["neighbor"] = [neigh_id, neighbor_azimuth]
                            break

            if is_neighbor:
                edge_info[(bs_id, neigh_id)] = azimuths_info
            else:
                neigh_G.remove_edges_from([[bs_id, neigh_id]])

    return neigh_G, edge_info

In [10]:
neigh_G, edge_info = find_real_neighbors(df, df_azimuth, del_G, pos)
neigh_G = distance_criterion_enhanced(neigh_G, pos, params=MEAN_DISTANCE_PARAMS, mean_distance_to_NN=mean_distances)
# neigh_G = angle_criterion(neigh_G, pos, min_angle=MIN_ANGLE, max_distance=None)
   

nodes - distance: 100%|██████████| 1457/1457 [00:00<00:00, 3577.76it/s]


In [11]:
len(neigh_G.edges())

2577

## Map creation

In [12]:
import folium
import numpy as np
from networkx import Graph
from pandas import DataFrame

In [13]:
def add_graph_edges(G_base: Graph, G: Graph, df: DataFrame, fg: folium.FeatureGroup, colour: str):
    for edge in G_base.edges:
        stations = []

        if(not(edge in G.edges)):
            stations.append(df.loc[edge[0], ['latitude', 'longitude']])
            stations.append(df.loc[edge[1], ['latitude', 'longitude']])

            folium.PolyLine(np.array(stations), color=colour, weight=2.5, opacity=1).add_to(fg)
     

In [23]:
tmp = edge_info.get(('0272290011', '0272290193'))
tmp['neighbor'][1]

105

In [24]:
# Function to generate edge info popup with visualization
def generate_edge_info_popup_with_visualization(edge, df, df_azimuth, edge_info):
    bs_id, neigh_id = edge
    bs_info = df.loc[bs_id]
    neigh_info = df.loc[neigh_id]

    azimuths_info = edge_info.get((bs_id, neigh_id), {"station": [None, None], "neighbor": [None, None]})
    unique_azimuths_bs = np.unique(df_azimuth.loc[df_azimuth['id_station_anfr'] == bs_id, 'angle_azimuth'])
    unique_azimuths_neigh = np.unique(df_azimuth.loc[df_azimuth['id_station_anfr'] == neigh_id, 'angle_azimuth'])

    popup_text = (
        f"<b>Base Station {bs_id} - {bs_info['nom_com']}</b><br>"
        f"Latitude: {bs_info['latitude']}, Longitude: {bs_info['longitude']}<br>"
        f"Azimuths: {'°, '.join(map(str, unique_azimuths_bs))}°<br>"
        f"Coverage Azimuths: {azimuths_info['station'][1]}°<br>"
        f"<br><b>Neighbor Station {neigh_id} - {neigh_info['nom_com']}</b><br>"
        f"Latitude: {neigh_info['latitude']}, Longitude: {neigh_info['longitude']}<br>"
        f"Azimuths: {'°, '.join(map(str, unique_azimuths_neigh))}°<br>"
        f"Coverage Azimuths: {azimuths_info['neighbor'][1]}°<br>"
    )

    return popup_text


In [25]:

# Function to add edges with interactive information
def add_graph_edges_with_visualization(G: Graph, df: DataFrame, df_azimuth: DataFrame, fg: folium.FeatureGroup, edge_info: dict):
    for edge in G.edges:
        stations = df.loc[[edge[0], edge[1]], ['latitude', 'longitude']].values
        popup_text = generate_edge_info_popup_with_visualization(edge, df, df_azimuth, edge_info)
        popup = folium.Popup(popup_text, max_width=300)
        line = folium.PolyLine(stations, color="#AAA662", weight=2.5, opacity=1)
        line.add_child(popup)
        line.add_to(fg)

In [26]:
# Function to add azimuth lines
def add_azimuth_lines(df: DataFrame, df_azimuth: DataFrame, fg: folium.FeatureGroup):
    azimuth_length = 0.01
    for bs_id in df.index:
        lat = df.loc[bs_id, 'latitude']
        long = df.loc[bs_id, 'longitude']
        for azimuth in df_azimuth.loc[df_azimuth['id_station_anfr'] == bs_id, 'angle_azimuth']:
            azimuth_angle = np.radians(azimuth)
            end_lat = lat + azimuth_length * np.cos(azimuth_angle)
            end_lon = long + azimuth_length * np.sin(azimuth_angle)
            line = folium.PolyLine([(lat, long), (end_lat, end_lon)], color='black', weight=2, opacity=0.7)
            popup_text = f"Azimuth: {azimuth}°"
            popup = folium.Popup(popup_text, max_width=150)
            line.add_child(popup)
            line.add_to(fg)


# Map visualization
def create_method_illustration_map(df: DataFrame, df_azimuth, del_graph: Graph, nei_graph: Graph, edge_info: dict, save_as: str, **kwargs):
    map = folium.Map(location=list(np.mean(df[['latitude', 'longitude']], axis=0)), zoom_start=8.5, tiles="Cartodb Positron")

    edges_del = folium.FeatureGroup(f"Edges - Delaunay triangulation ({len(del_graph.edges)})", show=True).add_to(map)
    edges_nei = folium.FeatureGroup(f"Edges - neighboring graph ({len(nei_graph.edges)})", show=True).add_to(map)
    azimuth_lines = folium.FeatureGroup(f"Azimuth Lines", show=True).add_to(map)

    add_graph_edges(del_graph, Graph(), df, edges_del, colour="lightblue")
    add_graph_edges_with_visualization(nei_graph, df, df_azimuth, edges_nei, edge_info)
    add_azimuth_lines(df, df_azimuth, azimuth_lines)

    points = folium.FeatureGroup(f"Base stations ({len(df)})").add_to(map)
    for bs_id in df.index:
        row = df.loc[bs_id]
        popup_text = (
            f"Station ID: {bs_id}<br>"
            f"Department: {row['nom_dep']}<br>"
            f"Commune: {row['nom_com']}<br>"
            f"Coordinates: ({row['latitude']}, {row['longitude']})<br>"
            f"2G: {row['site_2g']}<br>"
            f"3G: {row['site_3g']}<br>"
            f"4G: {row.get('site_4g', '1')}<br>"
            f"5G: {row['site_5g']}<br>"
        )
        station_data = df_azimuth.loc[df_azimuth['id_station_anfr'] == bs_id]
        for _, antenna in station_data.iterrows():
            popup_text += (
                f"<br>Antenna Frequency: {antenna['frequency_GHz']} GHz<br>"
                f"Antenna Azimuth: {antenna['angle_azimuth']}°<br>"
            )
        popup = folium.Popup(popup_text, max_width=200)
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            color='blue',
            radius=3,
            popup=popup,
            fillOpacity=1,
            fill=True
        ).add_to(points)

    folium.LayerControl().add_to(map)
    map.save(f"../../out/maps/neighbours_finding/{save_as}.html")

In [27]:
# Create and save map with visualization
create_method_illustration_map(df, df_azimuth, del_G, neigh_G, edge_info, save_as="anatoli_tmp")
